In [1]:
import numpy as np
from itertools import product

def compute_sigma_squared(snr_db):
    snr_linear = 10 ** (snr_db / 10)
    sigma_sq = 1 / (2 * snr_linear)
    return sigma_sq

def ym_find_optimal_theta_for_snr(sigma_sq):
    qpsk = np.array([1+1j, 1-1j, -1+1j, -1-1j]) / np.sqrt(2)
    N = len(qpsk)

    # Reference value at theta = 0
    total_0 = 0
    for k1, k2, i1, i2 in product(range(N), repeat=4):
        term = qpsk[k1] + qpsk[k2] - qpsk[i1] - qpsk[i2]
        exponent = -np.abs(term)**2 / (4 * sigma_sq)
        total_0 += np.exp(exponent)

    best_theta = 0
    best_total = float('inf')
    best_multiplicity = 0

    theta_values = np.arange(0, 180, 0.5)
    for theta in theta_values:

        theta_rad = np.deg2rad(theta)
        rotated_qpsk = qpsk * np.exp(1j * theta_rad)

        total = 0
        exp_values = []
        for k1, k2, i1, i2 in product(range(N), repeat=4):
            term = qpsk[k1] + rotated_qpsk[k2] - qpsk[i1] - rotated_qpsk[i2]
            exponent = -np.abs(term)**2 / (4 * sigma_sq)
            val = np.exp(exponent)
            exp_values.append(val)
            total += val

        min_exp_val = min(exp_values)
        multiplicity = exp_values.count(min_exp_val)

        if total < best_total:
            best_total = total
            best_theta = theta
            best_multiplicity = multiplicity

    improvement = total_0 - best_total
    return best_theta, best_multiplicity, improvement, total_0, best_total

def main():
    snr_db_range = np.arange(-2, 18, 2)
    results = []

    print("SNR(dB) | Best θ | Multiplicity | Improvement | θ=0 cost | Opt cost")
    print("---------------------------------------------------------------")
    for snr_db in snr_db_range:
        sigma_sq = compute_sigma_squared(snr_db)
        theta_opt, multiplicity, improvement, total_0, best_total = ym_find_optimal_theta_for_snr(sigma_sq)
        results.append((snr_db, theta_opt, multiplicity, improvement, total_0, best_total))
        print(f"{snr_db:7} | {theta_opt:6} | {multiplicity:12} | {improvement:11.6f} | {total_0:9.4f} | {best_total:9.4f}")

    return results

# Execute
if __name__ == "__main__":
    all_results = main()


SNR(dB) | Best θ | Multiplicity | Improvement | θ=0 cost | Opt cost
---------------------------------------------------------------
     -2 |   45.0 |            8 |    0.065328 |  108.5131 |  108.4478
      0 |  135.0 |            8 |    0.287772 |   80.6344 |   80.3466
      2 |   45.0 |            8 |    1.451419 |   58.4198 |   56.9684
      4 |   45.0 |            8 |    4.617397 |   44.2093 |   39.5919
      6 |  137.5 |            4 |    9.345167 |   37.8142 |   28.4690
      8 |   34.0 |            4 |   14.188754 |   36.1748 |   21.9861
     10 |  121.5 |            4 |   17.832499 |   36.0044 |   18.1719
     12 |   31.0 |            4 |   19.548079 |   36.0000 |   16.4519
     14 |   30.5 |            2 |   19.962313 |   36.0000 |   16.0377
     16 |   30.5 |            2 |   19.999259 |   36.0000 |   16.0007


In [2]:
def get_constellation(name):
    name = name.upper()
    if name == 'BPSK':
        return np.array([1, -1], dtype=complex)
    elif name == 'QPSK':
        return np.array([1+1j, 1-1j, -1+1j, -1-1j]) / np.sqrt(2)
    elif name == '8PSK':
        return np.exp(1j * 2 * np.pi * np.arange(8) / 8)
    elif name == '16PSK':
        return np.exp(1j * 2 * np.pi * np.arange(16) / 16)
    elif name == '8QAM':
        r1 = 1
        r2 = 2
        angles = np.deg2rad([0, 90, 180, 270])
        return np.concatenate([r1 * np.exp(1j * angles), r2 * np.exp(1j * (angles + np.pi/4))]) / np.sqrt(5)
    elif name == '16QAM':
        re = [-3, -1, 1, 3]
        im = [-3, -1, 1, 3]
        symbols = [complex(r, i) for r in re for i in im]
        return np.array(symbols) / np.sqrt(10)
    else:
        raise ValueError(f"Unsupported modulation: {name}")
def ym_find_optimal_theta_for_snr(sigma_sq, constellation, theta_step=0.5):
    N = len(constellation)

    total_0 = 0
    for k1, k2, i1, i2 in product(range(N), repeat=4):
        term = constellation[k1] + constellation[k2] - constellation[i1] - constellation[i2]
        exponent = -np.abs(term)**2 / (4 * sigma_sq)
        total_0 += np.exp(exponent)

    best_theta = 0
    best_total = float('inf')
    best_multiplicity = 0

    for theta in np.arange(0, 180, theta_step):
        theta_rad = np.deg2rad(theta)
        rotated = constellation * np.exp(1j * theta_rad)

        total = 0
        exp_values = []
        for k1, k2, i1, i2 in product(range(N), repeat=4):
            term = constellation[k1] + rotated[k2] - constellation[i1] - rotated[i2]
            exponent = -np.abs(term)**2 / (4 * sigma_sq)
            val = np.exp(exponent)
            exp_values.append(val)
            total += val

        min_exp_val = min(exp_values)
        multiplicity = exp_values.count(min_exp_val)

        if total < best_total:
            best_total = total
            best_theta = theta
            best_multiplicity = multiplicity
        elif np.isclose(total, best_total, atol=1e-10):
            best_theta = min(best_theta, theta)

    improvement = total_0 - best_total
    return best_theta, best_multiplicity, improvement, total_0, best_total
def main():
    modulations = ['BPSK', 'QPSK', '8PSK', '8QAM']
    snr_db_range = np.arange(-2, 18, 2)

    for mod in modulations:
        print(f"\n=== {mod} ===")
        constellation = get_constellation(mod)
        print("SNR(dB) | Best θ | Multiplicity | Improvement | θ=0 cost | Opt cost")
        print("---------------------------------------------------------------")
        for snr_db in snr_db_range:
            sigma_sq = compute_sigma_squared(snr_db)
            theta_opt, multiplicity, improvement, total_0, best_total = ym_find_optimal_theta_for_snr(sigma_sq, constellation)
            print(f"{snr_db:7} | {theta_opt:6.1f} | {multiplicity:12} | {improvement:11.6f} | {total_0:9.4f} | {best_total:9.4f}")
if __name__ == "__main__":
    main()



=== BPSK ===
SNR(dB) | Best θ | Multiplicity | Improvement | θ=0 cost | Opt cost
---------------------------------------------------------------
     -2 |   90.0 |            4 |    1.692240 |    8.2777 |    6.5855
      0 |   90.0 |            4 |    1.927408 |    7.0834 |    5.1559
      2 |   90.0 |            4 |    1.992946 |    6.3361 |    4.3432
      4 |   90.0 |            4 |    1.999827 |    6.0526 |    4.0528
      6 |   90.0 |            4 |    2.000000 |    6.0028 |    4.0028
      8 |   90.0 |            4 |    2.000000 |    6.0000 |    4.0000
     10 |   84.5 |            2 |    2.000000 |    6.0000 |    4.0000
     12 |   63.5 |            2 |    2.000000 |    6.0000 |    4.0000
     14 |   50.5 |            2 |    2.000000 |    6.0000 |    4.0000
     16 |   39.5 |            2 |    2.000000 |    6.0000 |    4.0000

=== QPSK ===
SNR(dB) | Best θ | Multiplicity | Improvement | θ=0 cost | Opt cost
---------------------------------------------------------------
     -2 

In [1]:
import numpy as np
from itertools import product
import random

def get_constellation(name):
    name = name.upper()
    if name == 'BPSK':
        return np.array([1, -1], dtype=complex)
    elif name == 'QPSK':
        return np.array([1+1j, 1-1j, -1+1j, -1-1j]) / np.sqrt(2)
    elif name == '8PSK':
        return np.exp(1j * 2 * np.pi * np.arange(8) / 8)
    elif name == '16PSK':
        return np.exp(1j * 2 * np.pi * np.arange(16) / 16)
    elif name == '8QAM':
        r1, r2 = 1, 2
        angles = np.deg2rad([0, 90, 180, 270])
        return np.concatenate([r1 * np.exp(1j * angles), r2 * np.exp(1j * (angles + np.pi/4))]) / np.sqrt(5)
    elif name == '16QAM':
        re = [-3, -1, 1, 3]
        im = [-3, -1, 1, 3]
        symbols = [complex(r, i) for r in re for i in im]
        return np.array(symbols) / np.sqrt(10)
    else:
        raise ValueError(f"Unsupported modulation: {name}")

def compute_sigma_squared(snr_db):
    snr_linear = 10 ** (snr_db / 10)
    return 1 / (2 * snr_linear)

def _iterate_or_sample_indices(N, num_samples=None):

    if num_samples is None:
        for tup in product(range(N), repeat=6):
            yield tup
    else:
        for _ in range(num_samples):
            yield (random.randrange(N), random.randrange(N),
                   random.randrange(N), random.randrange(N),
                   random.randrange(N), random.randrange(N))

def ym_find_optimal_thetas_for_snr_3users_rotate23(sigma_sq, constellation,
                                                 theta_step=1.0,
                                                 max_combinations_full=5_000_000,
                                                 sample_combinations=200_000):

    N = len(constellation)
    full_count = N ** 6
    sampled = False
    num_samples = None
    if full_count > max_combinations_full:
        sampled = True
        num_samples = sample_combinations

    total_0 = 0.0
    for k1, k2, k3, i1, i2, i3 in _iterate_or_sample_indices(N, num_samples=num_samples):
        term = (constellation[k1] + constellation[k2] + constellation[k3]
                - constellation[i1] - constellation[i2] - constellation[i3])
        exponent = -np.abs(term) ** 2 / (4 * sigma_sq)
        total_0 += np.exp(exponent)
    if sampled:
        total_0 = total_0 * (full_count / num_samples)

    best_total = float('inf')
    best_theta2 = 0.0
    best_theta3 = 0.0
    best_multiplicity = 0

    thetas = np.arange(0.0, 180.0, theta_step)

    C = constellation

    for theta2 in thetas:
        rot2 = C * np.exp(1j * np.deg2rad(theta2))
        for theta3 in thetas:
            rot3 = C * np.exp(1j * np.deg2rad(theta3))

            total = 0.0
            exp_values = []

            for k1, k2, k3, i1, i2, i3 in _iterate_or_sample_indices(N, num_samples=num_samples):
                term = (C[k1] + rot2[k2] + rot3[k3]
                        - C[i1] - rot2[i2] - rot3[i3])
                exponent = -np.abs(term) ** 2 / (4 * sigma_sq)
                val = np.exp(exponent)
                exp_values.append(val)
                total += val

            if sampled:
                total = total * (full_count / num_samples)
                min_exp_val = min(exp_values) if exp_values else 0.0
                multiplicity = sum(1 for v in exp_values if np.isclose(v, min_exp_val))
                multiplicity = int(multiplicity * (full_count / num_samples))
            else:
                min_exp_val = min(exp_values)
                multiplicity = exp_values.count(min_exp_val)

            if total < best_total - 1e-12:
                best_total = total
                best_theta2 = theta2
                best_theta3 = theta3
                best_multiplicity = multiplicity
            elif np.isclose(total, best_total, atol=1e-10):
                if (theta2, theta3) < (best_theta2, best_theta3):
                    best_theta2, best_theta3 = theta2, theta3

    improvement = total_0 - best_total
    return best_theta2, best_theta3, best_multiplicity, improvement, total_0, best_total, sampled

def main():
    modulations = ['BPSK', 'QPSK']
    snr_db_range = np.arange(-2, 18, 2)

    for mod in modulations:
        print(f"\n=== {mod} (3 users, rotate user-2 & user-3) ===")
        C = get_constellation(mod)
        print("SNR(dB) | Best θ2 | Best θ3 | Multiplicity | Improvement | θ=0 cost | Opt cost | sampled")
        print("-----------------------------------------------------------------------------------------")
        for snr_db in snr_db_range:
            sigma_sq = compute_sigma_squared(snr_db)
            theta2_opt, theta3_opt, multiplicity, improvement, total_0, best_total, sampled = \
                ym_find_optimal_thetas_for_snr_3users_rotate23(sigma_sq, C, theta_step=5.0,
                                                              max_combinations_full=5_000_000,
                                                              sample_combinations=200_000)
            print(f"{snr_db:7} | {theta2_opt:7.1f} | {theta3_opt:7.1f} | {multiplicity:12} | {improvement:11.6f} | {total_0:9.4f} | {best_total:9.4f} | {sampled}")

if __name__ == "__main__":
    main()



=== BPSK (3 users, rotate user-2 & user-3) ===
SNR(dB) | Best θ2 | Best θ3 | Multiplicity | Improvement | θ=0 cost | Opt cost | sampled
-----------------------------------------------------------------------------------------
     -2 |    60.0 |   120.0 |            6 |    8.067597 |   28.5705 |   20.5029 | False
      0 |    60.0 |   120.0 |            6 |    9.160256 |   24.0641 |   14.9038 | False
      2 |    45.0 |    95.0 |            2 |    9.853344 |   21.2604 |   11.4071 | False
      4 |    40.0 |    85.0 |            2 |   10.810758 |   20.1974 |    9.3866 | False
      6 |    40.0 |    80.0 |            2 |   11.599380 |   20.0105 |    8.4111 | False
      8 |    35.0 |    75.0 |            2 |   11.922442 |   20.0001 |    8.0777 | False
     10 |    35.0 |    75.0 |            2 |   11.994841 |   20.0000 |    8.0052 | False
     12 |    35.0 |    75.0 |            2 |   11.999923 |   20.0000 |    8.0001 | False
     14 |    35.0 |    75.0 |            2 |   12.000000 |   